In [ ]:
import pandas as pd
import numpy as np
import math
import nltk
from collections import Counter
from nltk.corpus import wordnet as w
random_state = 114514

In [ ]:
# Load data
gpt_data = pd.read_csv("data_process/gpt.csv")
human_data = pd.read_csv("data_process/human.csv")

# Combine the two datasets into one
data = pd.concat([gpt_data, human_data], ignore_index=True)

In [ ]:
# Feature extraction functions
def sentence_length(text):
    text = str(text)
    sentences = nltk.sent_tokenize(text)
    numberofsentences = len(sentences)
    total_words = 0
    for i in sentences:
        total_words += len(i.split())
    avg_sentence = total_words / numberofsentences
    return numberofsentences, avg_sentence

def repetitivewords(text):
    text = str(text)
    token = nltk.word_tokenize(text.lower())
    synsets = []
    for i in token:
        synsets.extend(w.synsets(i))
    synonyms = []
    for synset in synsets:
        synonyms.append([lemma.name() for lemma in synset.lemmas()])
    repeat = 0
    for index in range(len(synonyms)):
        for nextindex in range(index+1, len(synonyms)):
            if len(set(synonyms[index]) & set(synonyms[nextindex])) > 0:
                repeat += 1
    return repeat / len(token)

def entropy(text):
    text = str(text)
    tokens = nltk.word_tokenize(text.lower())
    tokennumber = Counter(tokens)
    total = len(tokens)
    numberofprobs = []
    for count in tokennumber.values():
        prob = count / total
        numberofprobs.append(prob)
    entropy = 0.0
    for i in numberofprobs:
        if i > 0:
            entropy -= i * (math.log(i, 2))
    return entropy

# Extract features
data['sent_length'], data['avg_sent_length'] = zip(*data['text'].apply(sentence_length))
data['repetitive_words'] = data['text'].apply(repetitivewords)
data['text_entropy'] = data['text'].apply(entropy)

# save the DataFrame to a CSV file
data.to_csv('data_process/processed_data.csv', index=False)

In [65]:
data = pd.read_csv("data_process/processed_data.csv")

In [66]:
data['generated'].value_counts()

generated
0    101643
1     10000
Name: count, dtype: int64

In [67]:
num_to_drop = data['generated'].value_counts()[0] - data['generated'].value_counts()[1]
data = data.drop(data[data['generated'] == 0].sample(n=num_to_drop).index)
data['generated'].value_counts()

generated
1    10000
0    10000
Name: count, dtype: int64

In [73]:
rows_to_keep = 6000
sampled_data = data.groupby('generated', group_keys=False).apply(lambda x: x.sample(n=rows_to_keep))
sampled_data.to_csv('data_process/processed_sample_data.csv', index=False)

### Sample gpt3 data with human data

In [1]:
import pandas as pd
import numpy as np
import math
import nltk
from collections import Counter
from nltk.corpus import wordnet as w
random_state = 114514
# Load data
rows_to_keep = 5000
gpt_data = pd.read_csv("data_process/gpt3.csv").sample(n=rows_to_keep, random_state=random_state)
human_data = pd.read_csv("data_process/human.csv").sample(n=rows_to_keep, random_state=random_state)

# Combine the two datasets into one
data = pd.concat([gpt_data, human_data], ignore_index=True)

# Feature extraction functions
def sentence_length(text):
    text = str(text)
    sentences = nltk.sent_tokenize(text)
    numberofsentences = len(sentences)
    total_words = 0
    for i in sentences:
        total_words += len(i.split())
    avg_sentence = total_words / numberofsentences
    return numberofsentences, avg_sentence

def repetitivewords(text):
    text = str(text)
    token = nltk.word_tokenize(text.lower())
    synsets = []
    for i in token:
        synsets.extend(w.synsets(i))
    synonyms = []
    for synset in synsets:
        synonyms.append([lemma.name() for lemma in synset.lemmas()])
    repeat = 0
    for index in range(len(synonyms)):
        for nextindex in range(index+1, len(synonyms)):
            if len(set(synonyms[index]) & set(synonyms[nextindex])) > 0:
                repeat += 1
    return repeat / len(token)

def entropy(text):
    text = str(text)
    tokens = nltk.word_tokenize(text.lower())
    tokennumber = Counter(tokens)
    total = len(tokens)
    numberofprobs = []
    for count in tokennumber.values():
        prob = count / total
        numberofprobs.append(prob)
    entropy = 0.0
    for i in numberofprobs:
        if i > 0:
            entropy -= i * (math.log(i, 2))
    return entropy

# Extract features
data['sent_length'], data['avg_sent_length'] = zip(*data['text'].apply(sentence_length))
data['repetitive_words'] = data['text'].apply(repetitivewords)
data['text_entropy'] = data['text'].apply(entropy)

# save the DataFrame to a CSV file
data.to_csv('data_process/processed_sample_data_gpt3.csv', index=False)

In [2]:
data

,text,generated
0,"As the weeks dragged on, Ghosn became increasi...",1
1,The Quadrantids meteor shower is known for its...,1
2,The case has also led to calls for greater int...,1
3,The tariffs have not only hurt American consum...,1
4,Question 3: What is the origin of the word “li...,1
...,...,...
9995,The man behind Trump’s favorite unproven treat...,0
9996,"Mr. Neumann, who has seven children and many m...",0
9997,"In a lengthy and impassioned dissent, Judge Jo...",0
9998,Institutions will face ethical and sometimes l...,0
